In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
owner = pd.read_csv("Owners.csv")
pet = pd.read_csv("Pets.csv")
procedure = pd.read_csv("ProceduresDetails.csv")
history= pd.read_csv("ProceduresHistory-1.csv")


In [4]:
history['Date'] = pd.to_datetime(history['Date'])
history['Day'] = history['Date'].dt.weekday_name
history.info()
owner.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2284 entries, 0 to 2283
Data columns (total 5 columns):
PetID               2284 non-null object
Date                2284 non-null datetime64[ns]
ProcedureType       2284 non-null object
ProcedureSubCode    2284 non-null int64
Day                 2284 non-null object
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 89.3+ KB


(89, 8)

In [5]:
pet.info()
pet.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
PetID      100 non-null object
Name       100 non-null object
Kind       100 non-null object
Gender     100 non-null object
Age        100 non-null int64
OwnerID    100 non-null int64
dtypes: int64(2), object(4)
memory usage: 4.8+ KB


(100, 6)

In [6]:
procedure.head(15)

,ProcedureType,ProcedureSubCode,Description,Price
0,OFFICE FEES,1,Office Call,32
1,OFFICE FEES,2,Emergency,100
2,OFFICE FEES,3,Reck,24
3,GROOMING,1,Bath,15
4,GROOMING,2,Flea Dip,15
5,GROOMING,3,Flea Spray,10
6,VACCINATIONS,1,Galaxie (DHLPP),15
7,VACCINATIONS,2,Leukemia,20
8,VACCINATIONS,3,Lyme,15
9,VACCINATIONS,4,PCR,15


In [7]:
history.head(7)

,PetID,Date,ProcedureType,ProcedureSubCode,Day
0,A8-1181,2016-01-10,VACCINATIONS,5,Sunday
1,E7-3766,2016-01-11,VACCINATIONS,5,Monday
2,B8-8740,2016-01-11,VACCINATIONS,5,Monday
3,D4-9443,2016-01-11,VACCINATIONS,5,Monday
4,F6-3398,2016-01-12,HOSPITALIZATION,1,Tuesday
5,E2-6642,2016-01-12,VACCINATIONS,5,Tuesday
6,A4-1165,2016-01-12,VACCINATIONS,5,Tuesday


### 1.join all

In [8]:
data = pd.merge(history,pet,how="inner",on="PetID")
data = pd.merge(data,owner,how="inner",on="OwnerID")
data = pd.merge(data,procedure,how="inner",on=["ProcedureType","ProcedureSubCode"])
data.shape
#data
data.head(5)



,PetID,Date,ProcedureType,ProcedureSubCode,Day,Name_x,Kind,Gender,Age,OwnerID,Name_y,Surname,StreetAddress,City,State,StateFull,ZipCode,Description,Price
0,T0-3277,2016-01-22,VACCINATIONS,5,Friday,Humbert,Cat,male,12,8133,Stacey,Randolph,1145 D Street,Southfield,MI,Michigan,48075,Rabies,10
1,G6-6501,2016-01-29,VACCINATIONS,5,Friday,Jake,Cat,male,2,3089,Lee,McKenzie,3295 Pinewood Avenue,Marquette,MI,Michigan,49855,Rabies,10
2,G6-6501,2016-07-30,VACCINATIONS,5,Saturday,Jake,Cat,male,2,3089,Lee,McKenzie,3295 Pinewood Avenue,Marquette,MI,Michigan,49855,Rabies,10
3,L4-6292,2016-02-05,VACCINATIONS,5,Friday,Bright,Dog,male,13,3089,Lee,McKenzie,3295 Pinewood Avenue,Marquette,MI,Michigan,49855,Rabies,10
4,L4-6292,2016-12-16,VACCINATIONS,5,Friday,Bright,Dog,male,13,3089,Lee,McKenzie,3295 Pinewood Avenue,Marquette,MI,Michigan,49855,Rabies,10


### 2. Procedure Type paling banyak

In [9]:
x = (data.groupby('ProcedureType')['ProcedureType'].count().max())
x
print(data['ProcedureType'].max(),x)

VACCINATIONS 29


### 3. Nama owner yg ke dokter lebih dari 3 kali

In [10]:
x = data.groupby(['OwnerID'])

data.groupby(['OwnerID']).filter(lambda x:len(x) > 3).groupby(['Name_y','Surname'])['PetID'].count()

Name_y  Surname 
Lee     McKenzie    4
Name: PetID, dtype: int64

### 4. Siapa yang paling sering ke dokter saat weekdays

In [11]:
notweekend = data[(data.Day != 'Saturday') | (data.Day != 'Sunday')] #memilih data history yang bukan weekend
weekday = notweekend.groupby('OwnerID')['OwnerID'].count().max() #menghitung data weekday dan mengelompokkan berdasarkan OwnerID kemudian dihitung jumlahnya
weekday
#print(data['Name_y'].max(),weekday)
#weekend['Name_y'].max()
notweekend.groupby(['OwnerID']).filter(lambda x:len(x) == weekday).groupby(['Name_y','Surname'])['PetID'].count()

Name_y  Surname 
Lee     McKenzie    4
Name: PetID, dtype: int64

### 5. Berapa total spending Jerry dan Jessica

In [12]:
# jj = ['Jerry','Jessica']
# jerry_jessica = data[data.Name_y.isin(jj)]
# jerry_jessica['Price'].sum()

data['Price'].where(data['Name_y'] == 'Jerry').sum()+data['Price'].where(data['Name_y'] == 'Jessica').sum()

425.0

### 6. Nama pet yang paling banyak di vaksin

In [13]:
vaksin = data[data.ProcedureType=='VACCINATIONS'].groupby('Name_x')['PetID'].count().reset_index()
max = vaksin['PetID'].max()
vaksin.loc[vaksin['PetID'] == max,['Name_x','PetID']]
#vaksin = data[data['ProcedureType']=='VACCINATIONS']
#data.groupby(['Name_x']).groups

,Name_x,PetID
2,Biscuit,3
4,Bonaparte,3


### 7. 5 anjing yang selisih umurnya paling jauh dibanding rata-rata umur anjing lain

In [14]:
data['Selisih'] = abs(data['Age']- data['Age'].mean())
data[data['Kind']=='Dog'].sort_values(by='Selisih',ascending=False).head(5)

,PetID,Date,ProcedureType,ProcedureSubCode,Day,Name_x,Kind,Gender,Age,OwnerID,Name_y,Surname,StreetAddress,City,State,StateFull,ZipCode,Description,Price,Selisih
32,Q9-1565,2016-05-08,GROOMING,3,Sunday,Scout,Dog,female,14,1766,Doris,Ray,1700 Mahlon Street,Farmington Hills,MI,Michigan,48335,Flea Spray,10,7.121951
29,P9-3625,2016-01-30,ORTHOPEDIC,7,Saturday,Cuddles,Dog,male,14,7579,Ricardo,Peterson,4963 Mahlon Street,Ann Arbor,MI,Michigan,48104,Pinning-I.M.,325,7.121951
33,P0-1725,2016-07-06,GROOMING,3,Wednesday,Lily,Dog,female,0,2419,Luisa,Cuellar,1308 Shingleton Road,Kalamazoo,MI,Michigan,49007,Flea Spray,10,6.878049
3,L4-6292,2016-02-05,VACCINATIONS,5,Friday,Bright,Dog,male,13,3089,Lee,McKenzie,3295 Pinewood Avenue,Marquette,MI,Michigan,49855,Rabies,10,6.121951
4,L4-6292,2016-12-16,VACCINATIONS,5,Friday,Bright,Dog,male,13,3089,Lee,McKenzie,3295 Pinewood Avenue,Marquette,MI,Michigan,49855,Rabies,10,6.121951


In [26]:
#drop redudansi data
dog =data[data.Kind=='Dog'][['PetID','Name_x','Kind','Age']].drop_duplicates()
# dog['Age'].mean()
dog['Selisih'] = abs(dog['Age']- dog['Age'].mean())
dog.sort_values(by='Selisih',ascending=False).head(5)

,PetID,Name_x,Kind,Age,Selisih
32,Q9-1565,Scout,Dog,14,7.111111
29,P9-3625,Cuddles,Dog,14,7.111111
33,P0-1725,Lily,Dog,0,6.888889
3,L4-6292,Bright,Dog,13,6.111111
21,P2-7342,Cuddles,Dog,13,6.111111


### 8. Binatang yang sering dibawa ke dokter saat weekend

In [15]:
weekend = data[(data.Day == 'Saturday') | (data.Day == 'Sunday')]
weekend1 = (weekend.groupby('Kind')['Kind'].count().max())
weekend.groupby(['Kind']).filter(lambda x:len(x) == weekend1).groupby(['Kind'])['PetID'].count()

Kind
Dog    12
Name: PetID, dtype: int64

### 9.Nama pet yang paling banyak di vaksin

In [16]:
vaksin = data[data.ProcedureType=='VACCINATIONS'].groupby('Name_x')['PetID'].count().reset_index()
max = vaksin['PetID'].max()
vaksin.loc[vaksin['PetID'] == max,['Name_x','PetID']]

,Name_x,PetID
2,Biscuit,3
4,Bonaparte,3


### 10.Rata-rata  harga paling mahal dari procedure

In [17]:
data_mean = data.groupby('ProcedureType').mean()
data_mean
data['Price'].groupby([data['ProcedureType']]).mean().sort_values(ascending=False).head(1)

ProcedureType
ORTHOPEDIC    267.5
Name: Price, dtype: float64